In [35]:
import spacy    
import unicodedata
import re

In [36]:
# Loads NLP English model
nlp = spacy.load('en')

In [37]:
oliver_testpaper = 'random_full.txt'#'data/texts/ebola/10.1007/s00705-020-04768-3.txt'

# Reads text
if 0:
    with open(oliver_testpaper, 'r') as file:
        text = unicodedata.normalize("NFKD", file.read().replace('\n', ' ')) # replaces \xa0 with " "
        text = re.sub(r'(?<=[.])(?=[^\s])', r' ', text) # spaces out concatenated sentences
else:
    text = "In Fig. 2 and Tables 3 (available online only), 4 (available online only), 5, we reprint the estimated CFR for each Ebola outbreak (by virus) and for Marburg virus. The Ebola Zaire virus is the most lethal with an overall estimated CFR ranging from 69 to 88%2,5,25,38,43,49,50 (Table 3 (available online only)). The CFR of outbreaks due to Ebola Sudan virus ranged from 53 to 69%1,24,51–53 (Table 4 (available online only)), and the CFR of outbreaks due to Ebola Bundibugyo ranged from 34 to 42%19,46,47 (Table 5). For the ongoing outbreak in West Africa due to Ebola Zaire, the estimated CFR, as measured among confirmed and probable cases with definitive outcome (recovered or fatal), is approximately 70%, and varies little among the three most affected countries (Guinea, Liberia and Sierra Leone; Table 6 (available online only) and Data Citation 2)38. The CFR among EVD cases reported by Nigeria (n=20) was 40%54). A second, unrelated EVD outbreak occurred in Équateur province, DRC between July and October 2014 resulting in 69 confirmed and probable cases with a CFR of 74%49. The CFR for Marburg is approximately 80%55–57)."
    
processed = False

In [38]:
def processText(text):
    global processed
    if not processed:
        processed = True
        return text.replace('%','% ').replace('-',' – ')
text = processText(text)
text

'In Fig. 2 and Tables 3 (available online only), 4 (available online only), 5, we reprint the estimated CFR for each Ebola outbreak (by virus) and for Marburg virus. The Ebola Zaire virus is the most lethal with an overall estimated CFR ranging from 69 to 88% 2,5,25,38,43,49,50 (Table 3 (available online only)). The CFR of outbreaks due to Ebola Sudan virus ranged from 53 to 69% 1,24,51–53 (Table 4 (available online only)), and the CFR of outbreaks due to Ebola Bundibugyo ranged from 34 to 42% 19,46,47 (Table 5). For the ongoing outbreak in West Africa due to Ebola Zaire, the estimated CFR, as measured among confirmed and probable cases with definitive outcome (recovered or fatal), is approximately 70% , and varies little among the three most affected countries (Guinea, Liberia and Sierra Leone; Table 6 (available online only) and Data Citation 2)38. The CFR among EVD cases reported by Nigeria (n=20) was 40% 54). A second, unrelated EVD outbreak occurred in Équateur province, DRC betwe

In [39]:
# Convert text into NLP object
textdoc = nlp(text)

In [40]:
# Named Entity Recognition
from spacy import displacy 
displacy.render(textdoc, style='ent')

In [41]:
sents = list(str(sent) for sent in list(textdoc.sents))

import pandas as pd
pd.set_option('display.max_colwidth', None)

# Put all sentences in the dataframe
sentsdf = pd.DataFrame(sents, columns = ['Sentence'])
sentsdf

,Sentence
0,"In Fig. 2 and Tables 3 (available online only), 4 (available online only), 5, we reprint the estimated CFR for each Ebola outbreak (by virus) and for Marburg virus."
1,"The Ebola Zaire virus is the most lethal with an overall estimated CFR ranging from 69 to 88% 2,5,25,38,43,49,50"
2,(Table 3 (available online only)).
3,"The CFR of outbreaks due to Ebola Sudan virus ranged from 53 to 69% 1,24,51–53 (Table 4 (available online only)), and the CFR of outbreaks due to Ebola Bundibugyo ranged from 34 to 42% 19,46,47 (Table 5)."
4,"For the ongoing outbreak in West Africa due to Ebola Zaire, the estimated CFR, as measured among confirmed and probable cases with definitive outcome (recovered or fatal), is approximately 70% , and varies little among the three most affected countries (Guinea, Liberia and Sierra Leone; Table 6 (available online only) and Data Citation 2)38."
5,The CFR among EVD cases reported by Nigeria (n=20) was 40% 54).
6,"A second, unrelated EVD outbreak occurred in Équateur province, DRC between July and October 2014 resulting in 69 confirmed and probable cases with a CFR of 74% 49."
7,The CFR for Marburg is approximately 80% 55–57).


In [42]:
# Listing keywords
numeric_keywords = ['distribution*','time','number*','ratio','proportion','period','±','total*','estimate*','%','review','parameter*','mean','period','value']
specific_keywords = ['CFR','r0','reproduct*''infections','death*','transmis*','laten*','contact','infectious','incubat*','casualties','mortal*','morbid*','outbreak*','epideme*']
contextual_keywords = ['GPE','DATE','TIME','PRODUCT']

# Creating a regular expression using keywords for searching and filtering 
numeric_regex = '|'.join(numeric_keywords)
specific_regex = '|'.join(specific_keywords)

trait_keywords = numeric_keywords + specific_keywords
trait_regex = '|'.join(trait_keywords)
trait_regex

numReg = re.compile(numeric_regex)
specReg = re.compile(specific_regex)
traitReg = re.compile(trait_regex)

In [43]:
import re

'''
Parses a sentence, looking for keywords.

args: sentence - string of sentence.
return: specific_keyword_count - total number of specific keywords in sentence
        numeric_keyword_count - total number of numeric keywords in sentence
        contextual_keyword_count - total number of contextual entity labels in sentence
        cardinality - total number of numeric entity labels in sentence
'''
def countKeywords(sentence):
    specific_keyword_count = numeric_keyword_count = contextual_keyword_count = cardinality = 0
    
    # counts specific and numeric keywords first
    for word in sentence.split():
        if re.match(specific_regex, word): specific_keyword_count += 1
        elif re.match(numeric_regex, word): numeric_keyword_count += 1
    
    # counts key entity labels
    sentence_obj = nlp(sentence)
    ent_labels = [ent.label_ for ent in sentence_obj.ents]
    for label in ent_labels:
        if label in contextual_keywords: contextual_keyword_count += 1
        elif label in ['CARDINAL']: cardinality += 1
        
    return specific_keyword_count, numeric_keyword_count, contextual_keyword_count, cardinality

In [44]:
sentsdf['SKC'],  sentsdf['NKC'], sentsdf['CKC'], sentsdf['Cardinality'] = zip(*sentsdf['Sentence'].apply(countKeywords))
sentsdf

,Sentence,SKC,NKC,CKC,Cardinality
0,"In Fig. 2 and Tables 3 (available online only), 4 (available online only), 5, we reprint the estimated CFR for each Ebola outbreak (by virus) and for Marburg virus.",1,1,1,3
1,"The Ebola Zaire virus is the most lethal with an overall estimated CFR ranging from 69 to 88% 2,5,25,38,43,49,50",0,1,0,1
2,(Table 3 (available online only)).,0,0,0,1
3,"The CFR of outbreaks due to Ebola Sudan virus ranged from 53 to 69% 1,24,51–53 (Table 4 (available online only)), and the CFR of outbreaks due to Ebola Bundibugyo ranged from 34 to 42% 19,46,47 (Table 5).",2,0,0,3
4,"For the ongoing outbreak in West Africa due to Ebola Zaire, the estimated CFR, as measured among confirmed and probable cases with definitive outcome (recovered or fatal), is approximately 70% , and varies little among the three most affected countries (Guinea, Liberia and Sierra Leone; Table 6 (available online only) and Data Citation 2)38.",1,1,3,3
5,The CFR among EVD cases reported by Nigeria (n=20) was 40% 54).,0,0,2,0
6,"A second, unrelated EVD outbreak occurred in Équateur province, DRC between July and October 2014 resulting in 69 confirmed and probable cases with a CFR of 74% 49.",1,0,3,1
7,The CFR for Marburg is approximately 80% 55–57).,0,0,1,0


In [45]:
# An example of filtering
sentsdf_filt = sentsdf.sort_values(by="Cardinality", ascending=False)
sentsdf_filt

,Sentence,SKC,NKC,CKC,Cardinality
0,"In Fig. 2 and Tables 3 (available online only), 4 (available online only), 5, we reprint the estimated CFR for each Ebola outbreak (by virus) and for Marburg virus.",1,1,1,3
3,"The CFR of outbreaks due to Ebola Sudan virus ranged from 53 to 69% 1,24,51–53 (Table 4 (available online only)), and the CFR of outbreaks due to Ebola Bundibugyo ranged from 34 to 42% 19,46,47 (Table 5).",2,0,0,3
4,"For the ongoing outbreak in West Africa due to Ebola Zaire, the estimated CFR, as measured among confirmed and probable cases with definitive outcome (recovered or fatal), is approximately 70% , and varies little among the three most affected countries (Guinea, Liberia and Sierra Leone; Table 6 (available online only) and Data Citation 2)38.",1,1,3,3
1,"The Ebola Zaire virus is the most lethal with an overall estimated CFR ranging from 69 to 88% 2,5,25,38,43,49,50",0,1,0,1
2,(Table 3 (available online only)).,0,0,0,1
6,"A second, unrelated EVD outbreak occurred in Équateur province, DRC between July and October 2014 resulting in 69 confirmed and probable cases with a CFR of 74% 49.",1,0,3,1
5,The CFR among EVD cases reported by Nigeria (n=20) was 40% 54).,0,0,2,0
7,The CFR for Marburg is approximately 80% 55–57).,0,0,1,0


In [46]:
# Sentences with most relevance
sentsdf_top = sentsdf.sort_values(by='SKC',ascending=False)
sentsdf_top.head()

,Sentence,SKC,NKC,CKC,Cardinality
3,"The CFR of outbreaks due to Ebola Sudan virus ranged from 53 to 69% 1,24,51–53 (Table 4 (available online only)), and the CFR of outbreaks due to Ebola Bundibugyo ranged from 34 to 42% 19,46,47 (Table 5).",2,0,0,3
0,"In Fig. 2 and Tables 3 (available online only), 4 (available online only), 5, we reprint the estimated CFR for each Ebola outbreak (by virus) and for Marburg virus.",1,1,1,3
4,"For the ongoing outbreak in West Africa due to Ebola Zaire, the estimated CFR, as measured among confirmed and probable cases with definitive outcome (recovered or fatal), is approximately 70% , and varies little among the three most affected countries (Guinea, Liberia and Sierra Leone; Table 6 (available online only) and Data Citation 2)38.",1,1,3,3
6,"A second, unrelated EVD outbreak occurred in Équateur province, DRC between July and October 2014 resulting in 69 confirmed and probable cases with a CFR of 74% 49.",1,0,3,1
1,"The Ebola Zaire virus is the most lethal with an overall estimated CFR ranging from 69 to 88% 2,5,25,38,43,49,50",0,1,0,1


In [47]:
'''
Displays the text surrounding the sentence with the provided index. To refer to the context, perhaps.
args: idx - index of sentence in question.
    df = Dataframe, default is sentsdf.
    pm = plus or minus for the indices of surrounding sentences.
'''
def context(idx, pm = 1, df = sentsdf):
    n = len(df)
    
    if (idx-pm) < 0 and (idx+pm) > n:
        return df
    elif (idx-pm) < 0:
        return df.loc[0:idx+pm,:]
    elif (idx-pm) > n:
        return df.loc[idx-pm:n,:]
    return df.loc[idx-pm:idx+pm,:]

from collections import Counter

'''
Print entity labels and their occurences within a given sentence.
args: sentence - The sentence in question.
    from_keywords - Whether or not to only include info on labels in the contextual keywords. Does not by default.
return: list containing tuples of entity labels and their occurences.
'''
def printEntityLabels(sentence, from_keywords = False):
    sentence_obj = nlp(sentence)
    ent_labels = [ent.label_ for ent in sentence_obj.ents if (not from_keywords or ent.label_ in contextual_keywords)]
    labels = Counter(ent_labels).keys()
    counts = Counter(ent_labels).values()
    return list(zip(labels, counts))

'''
Prints all sentences from Dataframe with provided keywords.
args: filter_words: Words to filter results by. By default, the trait keywords.
    df - The Dataframe to filter. By default, sentsdf.
return: Returns Dataframe with only sentences including keywords from the list.
'''
def sentencesWith(filter_words, df=sentsdf):
    if isinstance(filter_words, str):
        filter_regex = filter_words
    elif isinstance(filter_words, list):
        filter_regex = '|'.join(filter_words)
    return df[df.Sentence.str.lower().str.contains(filter_regex)]

'''
Calculates the relevance of a given dataframe, based on matches in the trait keyword  list. 
Used to determine relevance of the entire article.
args: df - The dataframe. By default, sentsdf.
return: The numeric approximation of the relevance of the provided dataframe.
'''
def calculateRelevance(df = sentsdf_top.head()):
    return sum(df.SKC) / len(df) * 10.000

In [48]:
calculateRelevance()

10.0

In [61]:
import numpy as np

f_out = open("testOutput.txt", "w")

dict_map = {
    "infectious":"infectious", 
    "contact":"contact",
    "latency":"latency",
    "latent":"latency",
    "reproduction":"reproduction",
    "incubation":"incubation",
    "r0":"reproduction",
    "fatalities":"deaths",
    "deaths":"deaths",
    "cases":"cases"
}
statsdf = pd.DataFrame([],columns=['Parameter','Estimates','n','Citation','Rule'])
print(statsdf)

from spacy.matcher import Matcher
bigMatcher = Matcher(nlp.vocab)
smallMatcher = Matcher(nlp.vocab)
smallMatcher.add("n_rule",None,[{'TEXT':'n'},{'TEXT':'='},{'LIKE_NUM':True}])
idx = None     

def find_n(doc):
    n_match = smallMatcher(doc)
    if n_match:
        match_id, start, end = n_match[0]
        return int(doc[end-1].text)
    return np.NaN
    
def funnel_values(estimates, parameter, rule_name, doc):
    global statsdf
    
    n = find_n(doc) #could make this faster
    # Parameter IS in dictionary
    if parameter:
        # Checking if estimates already included, in which case doesn't add a new row
        if not ((statsdf['Parameter'] == parameter) &\
             (statsdf['Estimates'] == estimates)).any():
            new_row = {'Parameter':parameter,\
                       'Estimates':estimates,\
                       'n':n,\
                       'Citation':idx,\
                       'Rule':rule_name}
            statsdf = statsdf.append(new_row,ignore_index=True)
         
    # Parameter is NOT in dictionary
    elif not (idx in list(statsdf.Citation) and\
        estimates in list(statsdf.Estimates.loc[statsdf.Citation == idx])):
        statsdf.loc[len(statsdf)] = [parameter, estimates, n, idx, rule_name]
        
def pm_map(matcher, doc, id, matches):
    for match_id, start, end in matches:
        string = str(doc[start:end])
        split_span = string.split()
        
        avg = round(float(split_span[-3]),2)
        moe = round(float(split_span[-1]),2)
        
        estimates = (avg-moe,avg+moe)
        parameter = dict_map.get(split_span[0])
        
        funnel_values(estimates, parameter, "pm_map", doc)

        
def bw_map(matcher, doc, id, matches):
    for match_id, start, end in matches:  
        lower = round(float(doc[start].text),2)
        upper = round(float(doc[end-1].text),2)
        
        estimates = (lower,upper)
        parameter = dict_map.get(str(doc[0]))
        #f_out.write(doc.text + "(SRC: BW_MAP)")
        
        funnel_values(estimates, parameter, "bw_map", doc)

# days_map finds matches for "# days", then searches the rest of the sentences' tokens for specific keywords to
# store as the corresponding parameter.
def num_map(matcher, doc, id, matches):
    for match_id, start, end in matches: 
        est_token = doc[end-2]
        estimate = est_token.text
        parameter = "None"
        for token in doc:
            closestIdx = abs(0 - est_token.i)
            # specReg is the regex for specific keywords
            if re.search(specReg, token.text.lower()):
                proximity_to_est = abs(token.i - est_token.i)
                # find the closest keyword to our estimate, as long as its within an arbitrary range (i said 15)
                if (proximity_to_est < closestIdx) & (proximity_to_est < 15):
                    closestIdx = token.i
                    parameter = token.text.lower()
        funnel_values(estimate, parameter, "days_map", doc)
        
# and_map works like days map, but for ranges specified with "and"
def and_map(matcher, doc, id, matches):
    for match_id, start, end in matches:
        lower_est_token = doc[start]
        lower = round(float(doc[start].text),2)
        upper = round(float(doc[end-1].text),2)
        
        estimates = (lower,upper)
        parameter = "None"
        for token in doc:
            closestIdx = abs(0 - lower_est_token.i)
            if re.search(specReg, token.text.lower()):
                proximity_to_est = abs(token.i - lower_est_token.i)
                if (proximity_to_est < closestIdx) & (proximity_to_est < 15):
                    closestIdx = token.i
                    parameter = token.text.lower()
        f_out.write(parameter)
        funnel_values(estimates, dict_map.get(parameter), "and_map", doc)

Empty DataFrame
Columns: [Parameter, Estimates, n, Citation, Rule]
Index: []


In [62]:
pm_rule = [{"IS_ALPHA":True},{"IS_ALPHA":True},{"IS_ALPHA":True},\
           {"LIKE_NUM":True}, {"TEXT":"±"}, {"LIKE_NUM":True}]
num_rule = [{"IS_ALPHA":True}, {"IS_ALPHA":True}, {"LIKE_NUM":True}, {"TEXT":{"REGEX":"day*|%"}}]

bigMatcher.add("pm_rule", pm_map, pm_rule)
bigMatcher.add("and_rule", and_map,\
               [{"LIKE_NUM":True}, {"TEXT":"and"}, {"LIKE_NUM":True}],\
               [{"LIKE_NUM":True}, {"TEXT":"–"}, {"LIKE_NUM":True}],\
               [{"LIKE_NUM":True}, {"TEXT":"to"}, {"LIKE_NUM":True}])
bigMatcher.add("num_rule", num_map, num_rule)

In [63]:
def cases_deaths_map(matcher, doc, id, matches):
    for match_id, start, end in matches:
        estimates = int(doc[start].text.replace(",",""))
        parameter = doc[end-1].text.replace("fatalities","deaths")
        funnel_values(estimates, parameter, "cases_deaths_map", doc)
        
bigMatcher.add("cases_deaths_rule", cases_deaths_map, 
               [{"LIKE_NUM":True},{"TEXT":{"REGEX":"fatalities|deaths|cases"}}])

In [64]:
statsdf = statsdf.iloc[0:0]

sents_filt = sentsdf
for idx in sents_filt.index:
    sentence = sents_filt.Sentence[idx]
    doc = nlp(sentence)
    matches = bigMatcher(doc)

In [65]:
# Order the Parameters column variables so that "None" is ranked lowest (for readability)
from pandas.api.types import CategoricalDtype

statsparams = list(statsdf.Parameter)
# If our set of parameters has no duplicates (does not apply to sample abstract, which stores values across countries)
# We probably should figure out how to separate the data from a paper if it deals with multiple countries?
if len(statsparams) == len(set(statsparams)):
    if ("None" in statsparams):
        statsparams.append(statsparams.pop(statsparams.index("None")))
    statsdf["Parameter"].astype(CategoricalDtype(categories=statsparams, ordered=True))
    # Sorted stats dataframe:
    statsdf = statsdf.sort_values('Parameter', ascending=False)

display(statsdf)
sentsdf.iloc[list(set(statsdf.Citation))]

,Parameter,Estimates,n,Citation,Rule
0,None,"(69.0, 88.0)",NaN,1,and_map
1,None,"(53.0, 69.0)",NaN,3,and_map
2,None,"(34.0, 42.0)",NaN,3,and_map
3,None,70,NaN,4,days_map
4,None,74,NaN,6,days_map
5,None,80,NaN,7,days_map


,Sentence,SKC,NKC,CKC,Cardinality
1,"The Ebola Zaire virus is the most lethal with an overall estimated CFR ranging from 69 to 88% 2,5,25,38,43,49,50",0,1,0,1
3,"The CFR of outbreaks due to Ebola Sudan virus ranged from 53 to 69% 1,24,51–53 (Table 4 (available online only)), and the CFR of outbreaks due to Ebola Bundibugyo ranged from 34 to 42% 19,46,47 (Table 5).",2,0,0,3
4,"For the ongoing outbreak in West Africa due to Ebola Zaire, the estimated CFR, as measured among confirmed and probable cases with definitive outcome (recovered or fatal), is approximately 70% , and varies little among the three most affected countries (Guinea, Liberia and Sierra Leone; Table 6 (available online only) and Data Citation 2)38.",1,1,3,3
6,"A second, unrelated EVD outbreak occurred in Équateur province, DRC between July and October 2014 resulting in 69 confirmed and probable cases with a CFR of 74% 49.",1,0,3,1
7,The CFR for Marburg is approximately 80% 55–57).,0,0,1,0


In [55]:
sents

['In Fig. 2 and Tables 3 (available online only), 4 (available online only), 5, we reprint the estimated CFR for each Ebola outbreak (by virus) and for Marburg virus.',
 'The Ebola Zaire virus is the most lethal with an overall estimated CFR ranging from 69 to 88% 2,5,25,38,43,49,50',
 '(Table 3 (available online only)).',
 'The CFR of outbreaks due to Ebola Sudan virus ranged from 53 to 69% 1,24,51–53 (Table 4 (available online only)), and the CFR of outbreaks due to Ebola Bundibugyo ranged from 34 to 42% 19,46,47 (Table 5).',
 'For the ongoing outbreak in West Africa due to Ebola Zaire, the estimated CFR, as measured among confirmed and probable cases with definitive outcome (recovered or fatal), is approximately 70% , and varies little among the three most affected countries (Guinea, Liberia and Sierra Leone; Table 6 (available online only) and Data Citation 2)38.',
 'The CFR among EVD cases reported by Nigeria (n=20) was 40% 54).',
 'A second, unrelated EVD outbreak occurred in Équ